![](https://live.staticflickr.com/65535/52663424849_8939aa98f9_k.jpg)

In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import mysql.connector as mysql
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
#connecting to the mysql server.
cnx = mysql.connect(
    host="localhost",
    user="root",
    passwd="0710",
    database="gdb041",
    raise_on_warnings=True
)
cursor=cnx.cursor()

In [ ]:
#printing all the table_names in the database gdb041
cursor.execute("show tables")
tables=cursor.fetchall()
for i in range(0,len(tables)):
    print(tables[i][0])

## Tables in the database (gdb041)

>dim_customer: customer_code, customer, platform, channel, market, sub_zone, region

contains the info about the different customers (Here the customer_code is the primary key. There are customers with many customer_codes.)

>dim_product: product_code, devision, segment, category, product, variant

contains the directory of all the products sold by the firm. (here the product_code is the primary key.)

>fact_forecast_monthly: date, fiscal_year, product_code, customer_code, forecast_quantity

contains the monthly forecast for each product_code (product_code, customer_code are the foregin keys)

>fact_freight_cost: market, fiscal_year,freight_pct, other_cost_pct

contains the freight and other cost of each market and fiscal_year. The freight and other cost are uniform for each product in the particular market and fiscal_year

>fact_gross_price: product_code, fiscal_year, gross_price

contains the gross_price of each product for each fiscal_year

>fact_manufacturing_cost: product_code, cost_year, manufacturing_cost

contains the manufacturing cost of each product for that particular fiscal_year

>fact_post_invoice_deductions: customer_code, product_code, date, discounts_pct, other_deductions_pct

contains the post invoice deductions like discounts on specific product and other deductions

>fact_pre_invoice_deductions: customer_code, fiscal_year, pre_invoice_discount_pct

contains the pre invoice deductions like special discount to a customer for strong relationships.

>fact_sales_monthly: date, product_code, customer_code, sold_quantity

contains the monthly sales data

### Questions

- Total monthly & yearly revenue by market, region, division
- Total net monthly & yearly profit by market, region, division
- Total deductions 
- Total sales for each fiscal year
- Which region is underperforming
- Revenue by fiscal year anf for the different regions
- How many customers we have in the lsat year and which customer generated the maximum revenue and also net income
- Contribution% in revenue of all the regions and total profit.
- Inventory Management from forecast and monthly (historical data)
- Profit by product category
- Top 5 customers & prodcuts, product_category using list comprehension


In [ ]:
temp_df=pd.read_sql("select * from dim_customer",cnx)
temp_df

In [ ]:
cursor.execute("select count(distinct customer) from dim_customer")
r1=cursor.fetchall()
cursor.execute("select count(distinct region) from dim_customer")
r2=cursor.fetchall()
print(r1[0][0],",",r2[0][0])

AtliQ Hardwares has a total of 74 different customers spanning over 4 different regions.

In [ ]:
temp_df=pd.read_sql("select * from dim_product",cnx)
temp_df

In [ ]:
#total revenue and sold quantity in each fiscal_year
query='''
with cte1 as (select year(date_add(date, interval 4 month)) as fiscal_year, product_code, customer_code, sold_quantity
from fact_sales_monthly)

select fiscal_year, round(sum(sold_quantity)/1000000,2) as total_sold_quantity_in_millions,
    round(sum(sold_quantity*gross_price)/1000000,2) as total_revenue_in_millions
from cte1 
join fact_gross_price
using(product_code, fiscal_year)
group by fiscal_year'''

df1=pd.read_sql(query,cnx)


In [ ]:
plt.plot(df1.fiscal_year,df1.total_revenue_in_millions)
plt.title('Total revenue in each fiscal year')
plt.xlabel('Year'); plt.ylabel('Revenue (In Millions)');
plt.xticks(df1.fiscal_year, df1.fiscal_year)
plt.show()

In [ ]:
plt.plot(df1.fiscal_year,df1.total_sold_quantity_in_millions)
plt.title('Total sold quantity in each fiscal year')
plt.xlabel('Fiscal_Year')
plt.ylabel('Total Quantity Sold (in Millions)')
plt.xticks(df1.fiscal_year,df1.fiscal_year)
plt.show()

In [ ]:
#total revenue and sold quantity in each month
query='''
with cte1 as (select date_format(date,"%b %y") as cdate, 
	date, year(date_add(date, interval 4 month)) as fiscal_year, product_code, customer_code, sold_quantity
	from fact_sales_monthly),
cte2 as (select cdate, date, fiscal_year, round(sum(sold_quantity)/1000000,2) as total_quantity_sold_in_millions, 
		round(sum(sold_quantity*gross_price)/1000000,2) as total_revenue_in_millions
	from cte1 
	join fact_gross_price
	using(product_code, fiscal_year)
	group by 1,2,3
	order by 2)

select cdate as Month, total_quantity_sold_in_millions, total_revenue_in_millions
from cte2
'''
df2=pd.read_sql(query,cnx)
df2.head()

In [ ]:
plt.plot(df2.Month,df2.total_revenue_in_millions)
#get current axes
ax = plt.gca()
#hide x-axis
ax.get_xaxis().set_visible(False)
plt.ylabel('Revenue for each month (in millions)')
plt.show()

In [ ]:
#created a view in db for monthly_revenue
query='''
ctreate 
with cte1 as (select *, year(date_add(date,interval 4 month)) as fiscal_year
from fact_sales_monthly)
select date, fiscal_year, product_code, customer_code, sold_quantity, sold_quantity*gross_price as revenue
from cte1
join fact_gross_price
using(product_code, fiscal_year)
'''
cursor.execute(query)
temp=cursor.fetchall()

In [ ]:
#revenue comparison for each market
query='''
select market, round(sum(revenue)/1000000,2) as total_revenue
from monthly_revenue
join dim_customer
using(customer_code)
group by 1
order by 2 desc
'''
df3=pd.read_sql(query,cnx)
df3.head()

In [ ]:
plt.bar(df3.market,df3.total_revenue)
plt.ylabel("Total Revenue (In Millions)")
plt.title("Total revenue by market")
plt.xticks(df3.market,df3.market,rotation=90)
plt.show()

In [ ]:
query1='''
select market, round(sum(revenue)/1000000,2) as total_revenue_2022
from monthly_revenue
join dim_customer
using(customer_code)
where fiscal_year=2022
group by 1
order by 2 desc
'''
query2='''
select market, round(sum(revenue)/1000000,2) as total_revenue_2021
from monthly_revenue
join dim_customer
using(customer_code)
where fiscal_year=2021
group by 1
order by 2 desc
'''
df4=pd.read_sql(query1,cnx)
df5=pd.read_sql(query2,cnx)
df4['total_revenue_2021']=df5.total_revenue_2021

In [ ]:
df4.head()

In [ ]:
#revenue comparison by markets for the fiscal_year 2022 and 2021
width=0.20
x = np.arange(len(df4.market))
plt.bar(x-width/2,df4.total_revenue_2021,width,label='2021')
plt.bar(x+width/2,df4.total_revenue_2022,width,label='2022')
plt.ylabel("Total Revenue (In Millions)")
plt.title("Total revenue by market in year 2021 & 2022")
plt.xticks(x,df4.market,rotation=90)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#lowest performers for the fiscal_year 2021 and 2022
df4.tail(5)

In [ ]:
#which products generated most revenue until now
query='''
select product, category, round(sum(revenue)/1000000,2) as total_revenue_mln,
	round(sum(sold_quantity)/1000,2) as total_quantity_thsnd
from monthly_revenue
join dim_product
using(product_code)
group by 1,2
order by 3 desc
'''
df5=pd.read_sql(query,cnx)
df5.head()

In [ ]:
#which product sold the most number of times and also least number of times
r1=df5[df5.total_quantity_thsnd==max(df5.total_quantity_thsnd)]
r2=df5[df5.total_quantity_thsnd==min(df5.total_quantity_thsnd)]
f=[r1,r2]
r3=pd.concat(f)
r3

In [ ]:
#which product sold most in the fiscal_year 2022
query='''
select product,category, round(sum(sold_quantity)/1000,2) as total_quantity_thsnd
from monthly_revenue
join dim_product
using(product_code)
where fiscal_year=2022
group by 1,2
order by 3 desc
limit 1
'''
cursor.execute(query)
r4=cursor.fetchall()
r4[0][0:2]

In [ ]:
#which product generated most revenue in the fiscal_year 2022
query='''
select product,category, round(sum(revenue)/1000000,2) as total_revenue_mln
from monthly_revenue
join dim_product
using(product_code)
where fiscal_year=2022
group by 1,2
order by 3 desc
limit 1
'''
cursor.execute(query)
r4=cursor.fetchall()
r4[0][0:3]

In [ ]:
#which product segement got sold most in total
query='''
select segment, round(sum(sold_quantity)/1000,2) as total_quantity_thsnd
from monthly_revenue
join dim_product
using(product_code)
group by 1
order by 2 desc
'''
df6=pd.read_sql(query,cnx)
df6

In [ ]:
fig = plt.figure()
fig.patch.set_facecolor('white')
palette_color=sns.color_palette('bright')
plt.rcParams['text.color'] = 'black'
plt.pie(df6.total_quantity_thsnd, labels=df6.segment, colors=palette_color, autopct='%.0f%%')
plt.title('Breakdown of total quantity sold by segment of products')
plt.show()

In [ ]:
#created a table for monthly net revenue after total deductions
query='''
create table `fact_net_profit`
with cte1 as (
select m.*,
	revenue*(r1.pre_invoice_discount_pct+r2.discounts_pct+r2.other_deductions_pct+r5.freight_pct+r5.other_cost_pct)+r3.manufacturing_cost as total_deduction
from monthly_revenue m
left join fact_pre_invoice_deductions r1
using(customer_code, fiscal_year)
left join fact_post_invoice_deductions r2
using(customer_code, product_code, date)
left join fact_manufacturing_cost r3
on m.product_code=r3.product_code and m.fiscal_year=r3.cost_year
left join dim_customer r4
using(customer_code)
left join fact_freight_cost r5
using(market, fiscal_year))

select *, revenue-total_deduction as net_revenue
from cte1
'''
cursor.execute(query)
temp=cursor.fetchall()

In [ ]:
#The total net revenue for each fiscal year

query='''
select fiscal_year, round(sum(net_revenue)/1000000,2) as total_profit
from fact_net_profit
group by 1
order by 1
'''
temp_df=pd.read_sql(query,cnx)
temp_df

In [ ]:
#comparison between the total revenue and total net revenue after all the deductions
df1['net_revenue_mln']=temp_df.total_profit
df1

In [ ]:
plt.plot(df1.fiscal_year,df1.total_revenue_in_millions,label='total revenue')
plt.plot(df1.fiscal_year,df1.net_revenue_mln,label='total profit')
plt.title('Total Revenue vs Profit in each fiscal year')
plt.xlabel('Year'); plt.ylabel('Revenue/Profit (In Millions)');
plt.xticks(df1.fiscal_year, df1.fiscal_year)
plt.legend()
plt.show()

In [ ]:
#which customer generated maximum revenue and maximum profit
query1='''
with cte1 as (select customer, sum(revenue) as revenue1
    from monthly_revenue
    join dim_customer
    using(customer_code)
    group by 1),
    cte2 as (select customer, dense_rank() over(order by revenue1 desc) as rnk
    from cte1)
select customer
from cte2
where rnk=1
'''

query2='''
with cte1 as (select customer, sum(net_revenue) as revenue1
    from fact_net_profit
    join dim_customer
    using(customer_code)
    group by 1),
    cte2 as (select customer, dense_rank() over(order by revenue1 desc) as rnk
    from cte1)
select customer
from cte2
where rnk=1
'''

cursor.execute(query1)
r5=cursor.fetchall()
cursor.execute(query2)
r6=cursor.fetchall()
print("The customer with maximum net revenue is {} and with maximum net profit is {}.".format(r5[0][0],r6[0][0]))

In [ ]:
#which customer generated maximum revenue and maximum profit in the fiscal_year 2020
query1='''
with cte1 as (select customer, sum(revenue) as revenue1, sum(net_revenue) as revenue2
    from fact_net_profit
    join dim_customer
    using(customer_code)
    where fiscal_year=2020
    group by 1),
    cte2 as (select customer, dense_rank() over(order by revenue1 desc) as rnk, dense_rank() over(order by revenue2 desc) as drnk
    from cte1)
select customer
from cte2
where rnk=1 or drnk=1
order by rnk
'''

cursor.execute(query1)
r5=cursor.fetchall()
print("The customer with maximum net revenue is {} and with maximum net profit is {}.".format(r5[0][0],r5[0][0]))

In [ ]:
#total contribution in profit by market

query='''
select market, round(sum(net_revenue)/1000000,2) as revenue1
from fact_net_profit
join dim_customer
using(customer_code)
where fiscal_year=2022
group by market
order by 1
'''
temp2=pd.read_sql(query,cnx)
temp2.head()

In [ ]:
plt.bar(temp2.market,temp2.revenue1)
plt.ylabel("Total Profit (In Millions)")
plt.title("Total Profit by market in fiscal year 2022")
plt.xticks(temp2.market,temp2.market,rotation=90)
plt.show()

In [ ]:
#total contribution in profit by product division in fiscal year 2022

query='''
select division, round(sum(net_revenue)/1000000,2) as revenue1
from fact_net_profit
join dim_product
using(product_code)
where fiscal_year=2022
group by 1
order by 1
'''
temp2=pd.read_sql(query,cnx)
temp2.head()

In [ ]:
fig = plt.figure()
fig.patch.set_facecolor('white')
palette_color=sns.color_palette('bright')
plt.rcParams['text.color'] = 'black'
plt.pie(temp2.revenue1, labels=temp2.division, colors=palette_color, autopct='%.0f%%')
plt.title('Breakdown of total profit by product division in 2022')
plt.show()

In [ ]:
# loss or no profit sections
query='''
select market,sum(abs(forecast_quantity-cast(sold_quantity as signed)))/sum(sold_quantity) as forecast_error_pct
from fact_sales_monthly
join fact_forecast_monthly
using (date, product_code, customer_code)
join dim_customer
using(customer_code)
group by 1
order by 1
'''
forecast_df=pd.read_sql(query,cnx)
forecast_df